In [2]:
from distutils.filelist import glob_to_re
import random
from skimage.feature import graycomatrix, graycoprops
import numpy as np
from pathlib import Path
import cv2
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc, accuracy_score, f1_score
from sklearn import preprocessing
from sklearn.utils import resample, shuffle
import pandas as pd


In [3]:
import os
os.chdir("..")

In [4]:
chall = "chall2"
glcm_feat = pd.read_feather(
    f"data/processed/features/glcm_features_{chall}.feather")
lbp_feat = pd.read_feather(
    f"data/processed/features/lbp_features_{chall}.feather")

glcm_lbp = pd.merge(glcm_feat,lbp_feat,on="image_0")
glcm_lbp = glcm_lbp.drop(columns=["index","image_0"])
y = glcm_lbp.pop("class")
X = glcm_lbp


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
svc = SVC(kernel='rbf', probability=True, class_weight='balanced', C=1.0)
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print(classification_report(y_test, y_pred))
print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))
print('\nAccuracy: ', accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.65      0.66      0.66       617
           1       0.80      0.67      0.73       801
           2       0.20      0.40      0.27       107

    accuracy                           0.65      1525
   macro avg       0.55      0.58      0.55      1525
weighted avg       0.69      0.65      0.67      1525

Confusion Matrix
 [[410 114  93]
 [182 539  80]
 [ 40  24  43]]

Accuracy:  0.6504918032786885


In [9]:
y_pred

array([1, 0, 0, ..., 1, 1, 0])

In [6]:
from xgboost import XGBClassifier

xgb_cl = XGBClassifier()

xgb_cl.fit(X_train, y_train)
y_pred = xgb_cl.predict(X_test)

print(classification_report(y_test, y_pred))
print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))
print('\nAccuracy: ', accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.70      0.72      0.71       617
           1       0.75      0.83      0.79       801
           2       0.38      0.03      0.05       107

    accuracy                           0.73      1525
   macro avg       0.61      0.53      0.52      1525
weighted avg       0.71      0.73      0.71      1525

Confusion Matrix
 [[442 172   3]
 [132 667   2]
 [ 59  45   3]]

Accuracy:  0.7291803278688525


In [7]:
bcc_meanshift = pd.read_feather("data/processed/chall2/train/bcc_color_features_meanshift_ull.feather")
mel_meanshift = pd.read_feather(
    "data/processed/chall2/train/mel_color_features_meanshift_ull.feather")

scc_meanshift = pd.read_feather(
    "data/processed/chall2/train/scc_color_features_meanshift_ull.feather")


bcc_raw = pd.read_feather(
    "data/processed/chall2/train/bcc_color_features_raw_ull.feather")
mel_raw = pd.read_feather(
    "data/processed/chall2/train/mel_color_features_raw_ull.feather")
scc_raw = pd.read_feather(
    "data/processed/chall2/train/scc_color_features_raw_ull.feather")

raw_dfs = pd.concat([bcc_raw, mel_raw, scc_raw])
meanshift_dfs = pd.concat([bcc_meanshift, mel_meanshift, scc_meanshift])


In [8]:
raw_dfs

,bgr_b_mean,bgr_b_std,bgr_b_skew,bgr_b_kurt,bgr_b_max,bgr_b_min,bgr_b_entrp,bgr_b_unq,bgr_g_mean,bgr_g_std,...,YCrCb_r_mean,YCrCb_r_std,YCrCb_r_skew,YCrCb_r_kurt,YCrCb_r_max,YCrCb_r_min,YCrCb_r_entrp,YCrCb_r_unq,name,image_0
0,172.711166,44.783367,-2.776359,6.949277,204.0,0.0,12.423009,205,155.500671,42.073441,...,9.196441,5.363695,0.479207,-0.076409,27.365009,-4.340816,-inf,3139,bcc01703,data/processed/chall2/train/bcc/bcc01703_inpai...
1,160.217026,19.615976,-4.524820,23.724243,175.0,1.0,12.466668,175,135.747375,21.083176,...,17.000277,5.394281,2.553683,8.542994,51.307377,-0.330772,-inf,3023,bcc01478,data/processed/chall2/train/bcc/bcc01478_inpai...
2,139.364456,83.024330,-0.818186,-1.088669,215.0,0.0,12.203903,216,77.367859,48.885403,...,29.083529,17.888847,-0.501741,-0.993032,72.582588,-0.330772,-inf,10931,bcc01798,data/processed/chall2/train/bcc/bcc01798_inpai...
3,183.222046,28.486214,-0.346837,-1.134958,225.0,92.0,12.464244,134,172.245560,34.390205,...,6.222847,4.842803,0.933852,1.542123,37.109802,-9.993786,-inf,4334,bcc00601,data/processed/chall2/train/bcc/bcc00601_inpai...
4,187.319611,12.583464,-1.460668,3.324144,207.0,86.0,12.474307,119,170.588425,23.204622,...,11.867117,8.063295,0.470359,-0.731111,37.065250,-3.172764,-inf,4162,bcc00435,data/processed/chall2/train/bcc/bcc00435_inpai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,206.706757,16.235504,-5.507888,35.371559,219.0,52.0,11.116156,167,175.643051,24.663004,...,14.736406,6.240335,2.599561,8.423414,48.009106,1.630823,11.047091,2127,scc00019,data/processed/chall2/train/scc/scc00019_inpai...
372,178.580185,25.900169,-4.292114,20.433192,199.0,1.0,12.461745,199,169.849854,26.421244,...,5.748914,6.738791,0.824343,-0.109810,28.358215,-16.932676,-inf,3738,scc00082,data/processed/chall2/train/scc/scc00082_inpai...
373,166.818436,71.371193,-1.456389,0.714020,241.0,0.0,12.330375,242,120.402267,58.768677,...,23.986753,13.817821,-0.265890,-0.824840,60.957981,-11.398708,-inf,16370,scc00362,data/processed/chall2/train/scc/scc00362_inpai...
374,201.277893,5.285622,-1.093860,3.685062,221.0,133.0,12.476300,66,146.996750,15.669910,...,28.550665,6.804332,-0.042472,-0.159755,58.533905,9.675723,12.447331,4532,scc00210,data/processed/chall2/train/scc/scc00210_inpai...


In [ ]:

import functools as ft


def get_chall2_class(path):
    if "bcc" in str(path):
        return "bcc"
    elif "mel" in str(path):
        return "mel"
    elif "scc" in str(path):
        return "scc"
    else:
        raise ValueError("class needs to be bcc, mel or scc")


dfs = [raw_dfs, glcm_feat, lbp_feat]
color_texture = ft.reduce(lambda left, right: pd.merge(left, right, on='image_0'), dfs)
# y = color_texture["image_0"].apply(get_chall2_class)
# color_texture = color_texture.drop(columns=['name', 'image_0','class'])
# X = color_texture

X[np.isnan(X)] = 0
X[np.isinf(X)] = 0


In [ ]:
color_texture.columns.values

array(['bgr_b_mean', 'bgr_b_std', 'bgr_b_skew', 'bgr_b_kurt', 'bgr_b_max',
       'bgr_b_min', 'bgr_b_entrp', 'bgr_b_unq', 'bgr_g_mean', 'bgr_g_std',
       'bgr_g_skew', 'bgr_g_kurt', 'bgr_g_max', 'bgr_g_min',
       'bgr_g_entrp', 'bgr_g_unq', 'bgr_r_mean', 'bgr_r_std',
       'bgr_r_skew', 'bgr_r_kurt', 'bgr_r_max', 'bgr_r_min',
       'bgr_r_entrp', 'bgr_r_unq', 'hsv_h_mean', 'hsv_h_std',
       'hsv_h_skew', 'hsv_h_kurt', 'hsv_h_max', 'hsv_h_min',
       'hsv_h_entrp', 'hsv_h_unq', 'hsv_s_mean', 'hsv_s_std',
       'hsv_s_skew', 'hsv_s_kurt', 'hsv_s_max', 'hsv_s_min',
       'hsv_s_entrp', 'hsv_s_unq', 'hsv_v_mean', 'hsv_v_std',
       'hsv_v_skew', 'hsv_v_kurt', 'hsv_v_max', 'hsv_v_min',
       'hsv_v_entrp', 'hsv_v_unq', 'lab_l_mean', 'lab_l_std',
       'lab_l_skew', 'lab_l_kurt', 'lab_l_max', 'lab_l_min',
       'lab_l_entrp', 'lab_l_unq', 'lab_a_mean', 'lab_a_std',
       'lab_a_skew', 'lab_a_kurt', 'lab_a_max', 'lab_a_min',
       'lab_a_entrp', 'lab_a_unq', 'lab_b_mean', 'l

In [11]:
len(X.columns)

215

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
svc = SVC(kernel='rbf', probability=True, class_weight='balanced', C=1.0)
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print(classification_report(y_test, y_pred))
print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))
print('\nAccuracy: ', accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.74      0.76      0.75       617
           1       0.91      0.80      0.85       786
           2       0.31      0.52      0.39       122

    accuracy                           0.76      1525
   macro avg       0.65      0.69      0.66      1525
weighted avg       0.79      0.76      0.77      1525

Confusion Matrix
 [[468  53  96]
 [114 630  42]
 [ 50   9  63]]

Accuracy:  0.761311475409836
